## LPI Pseudo Code
authors: Collin Bode, David Dralle   
email: collin@berkeley.edu   
date: 2020-06-17   
purpose:  to describe the algorithm used in the original SSR version of LPI.   
This ignores any GRASS GIS specific code and naming conventions.    


#### Import LiDAR files into raster with point count as value

- define target cell size: 2 m default
- For each tile convert to raster of same tilesize:
- raster 1: count number of ground points
- raster 2: count all points
- Mosaic all tiles together into two rasters

#### Define the size and position of the aggregation window

- cell size? 2 m default
- radius of effect based on model vegetation height? 22 m default
- box size in cells:  18 x 18 cells
- is the box symmetric? or does it shift with the seasons?  4 boxes defined to describe all 12 months. June used if no shift.

#### Neighborhood operation to sum points
 - LPI is calculated by running a neighborhood analysis on grids which contain a count of lidar points (ground filtered and raw - all points)   
 - The neighborhood analysis uses a bounding box defined by month (4 possible boxes) and sums all the lidar points within range.      
 - LPI = Ground Filtered sum of LiDAR points / Raw All summed Points.  Result is a dimensionless ratio of Canopy Gap or Openness. Collin tried several different equations, but in the end mostly just used the ratio.  Below are two.   

In [ ]:
if(year == 'ym4'):
        str_formula = "3.71 * ( A / B )^1.3455"
elif(year == 'yr4'):
        str_formula = "6.5 * ( A / B )^1.57 + 0.005"                                
else:
        str_formula = "A / B"
grass.run_command("r.mapcalculator", overwrite = ow, amap = pneighfilt, bmap = pneighunf, formula = str_formula, outfile = lpi)	

 - Ratios occasionally exceed 1.0 (100%), so additional cleaning step included to set those to 100%.   

In [ ]:
str_formula = "if( A > 1.0, 1.0, A)"
grass.run_command("r.mapcalculator", overwrite = ow, amap =  lpi, formula = str_formula, outfile = lpi)	

#### Export LPI as raster